In [ ]:
import cv2
import torch
import time
import numpy as np

In [ ]:
# Load a MiDas model for depth estimation
 
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)


In [ ]:
# Move model to GPU if available
module = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(module)
midas.eval()

In [ ]:
# Load transforms to resize and normalize the image
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.small_transform

In [ ]:
cap = cv2.VideoCapture("...") 
 # for using the primary webcam replace the brackets bt '0' if else replace them with 1 if we use the secondry webcam or specify the path of the video in the place of the dots...
cap

In [ ]:
 


while cap.isOpened():

    success, img = cap.read()

    start = time.time()

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply input transforms
    input_batch = transform(img).to(module)

    # Prediction and resize to original resolution
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()

    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)


    end = time.time()
    totalTime = end - start

    fps = 1 / totalTime

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    depth_map = (depth_map*255).astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map , cv2.COLORMAP_MAGMA)

    cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('Image', img)
    cv2.imshow('Depth Map', depth_map)

    #result = cv2.VideoWriter('filename.mp4',cv2.VideoWriter_fourcc(*'MJPG'),10)
    

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()